In [1]:
from langchain_core.documents import Document
from langchain_core.tools import tool
from typing import List, Any
from langchain_core.retrievers import BaseRetriever
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_core.callbacks import CallbackManagerForRetrieverRun
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

# Re-rank 모델
# uv add sentence-transformers
rerank_model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")
cross_reranker = CrossEncoderReranker(model=rerank_model, top_n=2)

# 웹 검색 리트리버 클래스 정의
class WebSearchRetriever(BaseRetriever):
    search_wrapper: DuckDuckGoSearchAPIWrapper

    def __init__(self, search_wrapper: DuckDuckGoSearchAPIWrapper, **kwargs: Any):
        super().__init__(search_wrapper=search_wrapper, **kwargs)

    def _get_relevant_documents(self, query: str, *, run_manager: CallbackManagerForRetrieverRun) -> List[Document]:
        results = self.search_wrapper.results(query, max_results=10)
        if not results:
            return [Document(page_content="관련 정보를 찾을 수 없습니다.")]
        
        formatted_docs = []
        for result in results:
            doc = Document(
                page_content=f'<Document href="{result.get("link", "")}"/>\n{result.get("snippet", "")}\n</Document>',
                metadata={
                    "source": "web search", 
                    "url": result.get("link", ""), 
                    "title": result.get("title", "")
                }
            )
            formatted_docs.append(doc)
        return formatted_docs

@tool
def web_search(query: str, search_period: str = 'm') -> List[Document]:
    """
    웹 검색을 수행하고 결과를 Document 리스트 형태로 반환하는 함수.
    search_period: 검색 기간 (d: 1일, w: 1주, m: 1달, y: 1년)
    """
    # WebSearchRetriever를 기반으로 ContextualCompressionRetriever를 초기화합니다.
    ddg_search_wrapper = DuckDuckGoSearchAPIWrapper(time=search_period)
    web_retriever = ContextualCompressionRetriever(
        base_compressor=cross_reranker, 
        base_retriever=WebSearchRetriever(search_wrapper=ddg_search_wrapper), 
    )

    docs = web_retriever.invoke(query)

    if len(docs) > 0:
        return docs
    
    return [Document(page_content="관련 정보를 찾을 수 없습니다.")]

config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

In [2]:
# 도구 목록을 정의 
tools = [web_search]

In [4]:
query = "KT 소액 결제 해킹 사건의 원인과 대책?"

tools[0].invoke(query)

[Document(metadata={'source': 'web search', 'url': 'https://viabeelo.tistory.com/entry/KT-소액결제-해킹사건-원인과-예방방법-총정리', 'title': 'KT 소액결제 해킹사건, 원인과 예방방법 총정리'}, page_content='<Document href="https://viabeelo.tistory.com/entry/KT-소액결제-해킹사건-원인과-예방방법-총정리"/>\nSep 20, 2025 · 이번 글에서는 KT 소액 결제 해킹 사건 의 원인 과 수법, 그리고 개인 피해를 예방할 수 있는 실질적인 방법을 정리해 드리겠습니다.\n</Document>'),
 Document(metadata={'source': 'web search', 'url': 'https://soso-cho.com/entry/KT-해킹-사건-전말-분석-소액결제-피해와-서버-침해-개인정보-유출-위험까지', 'title': 'KT 해킹 사건 전말 분석: 소액결제 피해와 서버 침해, 개인정보 유출 ...'}, page_content='<Document href="https://soso-cho.com/entry/KT-해킹-사건-전말-분석-소액결제-피해와-서버-침해-개인정보-유출-위험까지"/>\nSep 20, 2025 · KT 해킹 사건 은 단순한 기술적 사고를 넘어 사회 전반에 불안을 확산시키고 있습니다. 소액 결제 피해에서 시작된 사건은 곧 서버 침해 사실로 이어졌고, 개인정보 유출 가능성까지 제기되면서 국민들이 느끼는 불안은 더욱 커졌습니다.\n</Document>')]